# Практическое занятие

В рамках данного демо будем работать с синтетическим минибелковым ингибитором LCB3:

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7857403/pdf/370_426.pdf

Структура комплекса LCB3 с в контакте RBD доменом коронавируса:

https://www.rcsb.org/structure/7JZM

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time

%matplotlib inline
plt.ion()

In [2]:
! wget https://files.rcsb.org/download/7JZM.pdb # Скачаем файл со структурой

--2024-08-28 10:45:14--  https://files.rcsb.org/download/7JZM.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.159.157
Connecting to files.rcsb.org (files.rcsb.org)|128.6.159.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘7JZM.pdb’

7JZM.pdb                [  <=>               ] 291.96K  1.07MB/s    in 0.3s    

2024-08-28 10:45:15 (1.07 MB/s) - ‘7JZM.pdb’ saved [298971]



Для визуализации можно использовать как и десктопные приложения, такие как PyMOL или Chimera. На сайте rcsb также присутствует интерактивный viewer трехмерных структур. В питоновских ноутбуках, однако, могут пригодиться адаптированные библиотеки, например nglview или py3Dmol.

In [3]:
!pip install py3Dmol
import py3Dmol

In [6]:
p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

p.addModel(open('/content/7JZM.pdb','r').read(),'pdb') # добавить модель из файла

p.setStyle({'cartoon': {'color':'green'}}) # отобразить все в виде cartoon и покрасить в зеленый
p.setStyle({'chain': 'A'}, {'cartoon': {'color': 'blue'}})

p.zoomTo() # центрировать структуру

p.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
# Color specific residues in chain B purple and show them as sticks
residues = [420, 403, 417, 505]
for res in residues:
    p.setStyle({'chain': 'B', 'resi': res}, {
        'cartoon': {'color': 'red'},
        'stick': {'colorscheme': 'redCarbon'}
    })
# Center and display the structure
p.zoomTo()
p.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
import py3Dmol

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
view.addModel(open('/content/7JZM.pdb','r').read(),'pdb') # Add model from file
view.setStyle({'cartoon': {'color':'green'}}) # Display all in cartoon and color green
view.setStyle({'chain': 'A'}, {'stick': {'color': 'blue'}}) # Color chain A in red

# Highlight residues in chain A that are within 6 angstroms of residues 403, 420, 417, 505 in chain B
view.addStyle(
    {
        'and': [
            {'within': {
                'distance': 5,
                'sel': {
                    'or': [
                        {'resi': 403, 'chain': 'B'},
                        {'resi': 420, 'chain': 'B'},
                        {'resi': 417, 'chain': 'B'},
                        {'resi': 505, 'chain': 'B'}
                    ]
                }
            }},
            {'chain': 'A'}  # Only consider residues from chain A
        ]
    },
    {
        'stick': {
            'colorscheme': 'redCarbon'
        }
    }
)

view.zoomTo() # Center the structure
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# ProteinMPNN

In [9]:
!mkdir INPUT/
!mkdir OUTPUT_MPNN/
!mv 7JZM.pdb INPUT/

Устанавливаем ProteinMPNN локально.

In [10]:
!git clone https://github.com/dauparas/ProteinMPNN

Cloning into 'ProteinMPNN'...
remote: Enumerating objects: 634, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 634 (delta 185), reused 154 (delta 154), pack-reused 395 (from 1)
Receiving objects: 100% (634/634), 119.91 MiB | 18.63 MiB/s, done.
Resolving deltas: 100% (285/285), done.


Запустим ProteinMPNN для редизайна последовательности минипротеина.

In [11]:
%%bash

python ProteinMPNN/helper_scripts/parse_multiple_chains.py --input_path=INPUT/ --output_path=parsed_chains

python ProteinMPNN/helper_scripts/assign_fixed_chains.py --input_path=parsed_chains --output_path=assigned_chains --chain_list "A"

python ProteinMPNN/protein_mpnn_run.py \
        --jsonl_path parsed_chains \
        --chain_id_jsonl assigned_chains \
        --out_folder OUTPUT_MPNN \
        --num_seq_per_target 20 \
        --sampling_temp "0.1" \
        --seed 42 \
        --batch_size 1

----------------------------------------
fixed_positions_jsonl is NOT loaded
----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
bias_AA_jsonl is NOT loaded
----------------------------------------
tied_positions_jsonl is NOT loaded
----------------------------------------
bias by residue dictionary is not loaded, or not provided
----------------------------------------
discarded {'bad_chars': 0, 'too_long': 0, 'bad_seq_length': 0}
----------------------------------------
Number of edges: 48
Training noise level: 0.2A
Generating sequences for: 7JZM
20 sequences of length 258 generated in 59.5377 seconds


/content/ProteinMPNN/protein_mpnn_run.py:178: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Попробуем посмотреть статистики по аминокислотам.

Необходимо построить ЛОГО и вывести частоты аминокислот.

In [12]:
!pip install logomaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 80.3 MB/s eta 0:00:00


In [13]:
import logomaker as lm

In [14]:
with open('OUTPUT_MPNN/seqs/7JZM.fa', 'r') as f:
    raw_seqs = f.readlines()

In [15]:
# Достанем из .fa файла сами последовательности, без хедеров
seqs = [... for seq in raw_seqs if ... not in seq]
print('There are %d sequences, all of length %d'%(len(seqs), len(seqs[0])))
seqs[:5]

TypeError: 'in <string>' requires string as left operand, not ellipsis

In [ ]:
counts_mat = lm.alignment_to_matrix(seqs)
counts_mat.head()

In [ ]:
lm.Logo(counts_mat, figsize=(50, 5))

Можем ли сказать, что определенные позиции имеют консервативные аминокислоты? Совпадают ли в этих позициях предложенные ProteinMPNN с изначальной последовательность минипротеина?

# RFDiffusion

Основной колаб для работы с пайплайном:
https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/rf/examples/diffusion.ipynb#scrollTo=tVAE0BrnZoRR

AlphaFold 2 отдельно:
https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb#scrollTo=G4yBrceuFbf3